In [2]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
pwd

'/coc/scratch/sanisetty3/music_motion/motion_vqvae'

In [5]:
import wandb

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader

from torch.utils import data


import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator,VQFullMotionDataset
from einops import rearrange, reduce, pack, unpack
import librosa

In [6]:
from utils.motion_process import recover_from_ric
import visualize.plot_3d_global as plot_3d
from glob import glob
def to_xyz(motion, mean ,std , j = 22):
    motion_xyz = recover_from_ric(motion.cpu().float()*std+mean, j)
    motion_xyz = motion_xyz.reshape(motion.shape[0],-1, j, 3)
    return motion_xyz

            
def sample_render(motion_xyz , name , save_path , fps = 20):
    print(f"render start")
    
    gt_pose_vis = plot_3d.draw_to_batch(motion_xyz.numpy(),None, [os.path.join(save_path,name + ".gif")] , fps = fps)



In [ ]:
motion =  np.load("/srv/scratch/sanisetty3/music_motion/Choreomaster/new_joint_vecs/swap2.npy")[0]

In [19]:
motion = motion[:100]

In [40]:
motion.shape

(100, 22, 3)

In [20]:
sample_render(torch.Tensor(motion)[None,...] , "sample" , "/srv/scratch/sanisetty3/music_motion/Choreomaster/")

render start


In [24]:
from render_final import saveSMPL, render, hml2aist

In [39]:
saveSMPL((motion) , "/srv/scratch/sanisetty3/music_motion/Choreomaster/" , name = "sample")

cuda:0
./body_models/
Running SMPLify, it may take a few minutes.


/coc/scratch/sanisetty3/music_motion/motion_vqvae/visualize/simplify_loc2rot.py:111: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  root_loc = torch.tensor(keypoints_3d[:, 0])  # [bs, 3]
/coc/scratch/sanisetty3/music_motion/motion_vqvae/render_final.py:204: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vertices, rotations, global_orient, out, x_translations = rot2xyz(torch.tensor(motion_tensor).clone(), mask=None,


torch.Size([1, 25, 6, 100]) dict_keys(['pose', 'betas', 'cam'])
torch.Size([1, 6890, 3, 100])


In [24]:
aist = saveSMPL(motion)

cuda:0
./body_models/
Running SMPLify, it may take a few minutes.
torch.Size([100, 3, 3]) torch.Size([100, 23, 3, 3]) torch.Size([1, 3, 100])


/coc/scratch/sanisetty3/music_motion/motion_vqvae/visualize/simplify_loc2rot.py:111: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  root_loc = torch.tensor(keypoints_3d[:, 0])  # [bs, 3]
/coc/scratch/sanisetty3/music_motion/motion_vqvae/render_final.py:243: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vertices, rotations, global_orient, out, x_translations = rot2xyz(torch.tensor(motion_tensor).clone(), mask=None,


In [27]:
np.save("./choreo.npy" , aist)

In [36]:
aist[: , 6:9]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.03268224e-04, -1.32417679e-03, -1.30845408e-03],
       [-8.72833771e-04, -2.70038843e-03, -2.91233556e-03],
       [-3.37918196e-03, -3.99845839e-03, -5.17676538e-03],
       [-8.34951270e-03, -4.77743149e-03, -7.74157653e-03],
       [-1.47393998e-02, -4.20612097e-03, -1.04799978e-02],
       [-2.12891866e-02, -4.98509407e-03, -1.18798027e-02],
       [-2.59663202e-02, -5.60820103e-03, -1.23881474e-02],
       [-3.08203064e-02, -6.02364540e-03, -1.23435110e-02],
       [-4.53799926e-02, -5.08898497e-03, -1.20260268e-02],
       [-6.84565082e-02, -3.37529182e-03, -9.70230810e-03],
       [-8.95968601e-02, -2.05117464e-03, -7.60054216e-03],
       [-9.87266749e-02, -1.66171789e-03, -7.67991273e-03],
       [-9.44291875e-02, -1.97327137e-03, -1.02918949e-02],
       [-9.24920514e-02, -1.97327137e-03, -1.25371879e-02],
       [-9.19149220e-02, -1.97327137e-03, -1.33888284e-02],
       [-9.16058049e-02, -2.18105316e-03

In [34]:
motion2 =  np.load("/srv/scratch/sanisetty3/music_motion/AIST/new_joint_vecs/M_gBR_sBM_cAll_d04_mBR0_ch01.npy")

In [37]:
motion2[:,6:9]

array([[ 5.73869422e-03, -5.01594469e-02,  8.35798442e-01],
       [ 2.03748271e-02, -5.68524189e-02,  8.41108620e-01],
       [ 1.66292936e-02, -5.60050085e-02,  8.50593805e-01],
       [ 1.11118499e-02, -5.44041358e-02,  8.56597066e-01],
       [ 6.15976751e-03, -5.22650443e-02,  8.59110832e-01],
       [-1.16317347e-03, -5.05819619e-02,  8.59549880e-01],
       [-7.89534487e-03, -4.88173328e-02,  8.59025121e-01],
       [-1.13588367e-02, -4.87811007e-02,  8.67782295e-01],
       [-1.40893571e-02, -5.02511859e-02,  8.84514868e-01],
       [-2.14134883e-02, -5.06952666e-02,  9.03409600e-01],
       [-3.09906565e-02, -5.13617247e-02,  9.20028806e-01],
       [-3.75243612e-02, -5.19232973e-02,  9.33208227e-01],
       [-4.04615365e-02, -5.52401766e-02,  9.30880010e-01],
       [-4.56814878e-02, -5.39702214e-02,  9.13914382e-01],
       [-4.78381142e-02, -5.33165857e-02,  9.09768760e-01],
       [-5.10953292e-02, -5.41080311e-02,  9.18235481e-01],
       [-5.29866517e-02, -5.57096377e-02

In [52]:
def uniform(shape, min = 0, max = 1, device = None):
    return torch.zeros(shape, device = device).float().uniform_(0, 1)

def cosine_schedule(t):
    return torch.cos(t * math.pi * 0.5)
import math

In [53]:
batch  =4
seq_len  =128

In [54]:
rand_time = uniform((batch,))
rand_mask_probs = cosine_schedule(rand_time)
num_token_masked = (seq_len * rand_mask_probs).round().clamp(min = 1)

In [55]:
rand_time

tensor([0.7379, 0.7339, 0.2254, 0.0212])

In [56]:
rand_mask_probs

tensor([0.4001, 0.4059, 0.9380, 0.9994])

In [57]:
num_token_masked

tensor([ 51.,  52., 120., 128.])

In [58]:
from einops import rearrange, repeat

In [68]:
batch_randperm = torch.rand((batch, seq_len)).argsort(dim = -1)
mask = batch_randperm < rearrange(num_token_masked, 'b -> b 1')


In [97]:
scores = torch.zeros((4,128))
ids = torch.full((4,128), 1024, dtype = torch.long)

In [98]:
rand_mask_prob = cosine_schedule(torch.Tensor([1]))
num_token_masked = max(int((rand_mask_prob * seq_len).item()), 1)

masked_indices = scores.topk(num_token_masked, dim = -1).indices


In [99]:
ids = ids.scatter(1, masked_indices, 1024)

In [100]:
ids

tensor([[1024,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [1024,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0, 

In [86]:
rand_mask_prob

tensor([-4.3711e-08])

In [87]:
num_token_masked

1

In [ ]:
scores.topk()

In [101]:
pwd

'/coc/scratch/sanisetty3/music_motion/motion_vqvae'

### Everybody dance now

In [1]:
from tqdm import tqdm

In [2]:
# !pip install --upgrade --force-reinstall git+https://github.com/ytdl-org/youtube-dl.git

In [3]:
# Output Directories
output_folder = "../Choreomaster/vids/"

In [4]:
with open('../Choreomaster/everybodydancenow.txt') as f:
    urllist = [(line.strip()) for line in f.readlines()]

In [5]:
for url in tqdm(urllist[:10]):
    !youtube-dl --output "{output_folder}/%(id)s.%(ext)s" "{url}"

  0%|          | 0/10 [00:00<?, ?it/s]

[youtube] jyTpBWCgC8w: Downloading webpage
[download] ../Choreomaster/vids//jyTpBWCgC8w.webm has already been downloaded and merged


 10%|█         | 1/10 [00:01<00:14,  1.59s/it]

[youtube] GoHYQrFoM_c: Downloading webpage
[download] ../Choreomaster/vids//GoHYQrFoM_c.webm has already been downloaded and merged


 20%|██        | 2/10 [00:05<00:17,  2.16s/it]

...

 80%|████████  | 8/10 [00:52<00:11,  5.99s/it]

[youtube] TusVxcTzcGc: Downloading webpage
[dashsegments] Total fragments: 4
[download] Destination: ../Choreomaster/vids//TusVxcTzcGc.f303.webm
[download] 100% of 39.77MiB in 00:02.27MiB/s ETA 00:00706
[dashsegments] Total fragments: 1
[download] Destination: ../Choreomaster/vids//TusVxcTzcGc.f251.webm
[download] 100% of 2.09MiB in 00:00.81MiB/s ETA 00:002
[ffmpeg] Merging formats into "../Choreomaster/vids//TusVxcTzcGc.webm"
Deleting original file ../Choreomaster/vids//TusVxcTzcGc.f303.webm (pass -k to keep)
Deleting original file ../Choreomaster/vids//TusVxcTzcGc.f251.webm (pass -k to keep)


 90%|█████████ | 9/10 [01:00<00:06,  6.40s/it]

[youtube] _9KxZiNM6Wk: Downloading webpage
[dashsegments] Total fragments: 8
[download] Destination: ../Choreomaster/vids//_9KxZiNM6Wk.f137.mp4
[download] 100% of 72.25MiB in 00:29.43MiB/s ETA 00:00323
[dashsegments] Total fragments: 1
[download] Destination: ../Choreomaster/vids//_9KxZiNM6Wk.f251.webm
[download] 100% of 4.32MiB in 00:02.22MiB/s ETA 00:005
[ffmpeg] Merging formats into "../Choreomaster/vids//_9KxZiNM6Wk.mkv"
Deleting original file ../Choreomaster/vids//_9KxZiNM6Wk.f137.mp4 (pass -k to keep)
Deleting original file ../Choreomaster/vids//_9KxZiNM6Wk.f251.webm (pass -k to keep)


100%|██████████| 10/10 [01:35<00:00,  9.51s/it]


In [12]:
!python setup.py develop

running develop
running egg_info
creating virtualmarker.egg-info
writing virtualmarker.egg-info/PKG-INFO
writing dependency_links to virtualmarker.egg-info/dependency_links.txt
writing top-level names to virtualmarker.egg-info/top_level.txt
writing manifest file 'virtualmarker.egg-info/SOURCES.txt'
reading manifest file 'virtualmarker.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'virtualmarker.egg-info/SOURCES.txt'
running build_ext
Creating /coc/pskynet2/sanisetty3/miniconda3/envs/ai-choreo/lib/python3.7/site-packages/virtualmarker.egg-link (link to .)
Adding virtualmarker 0.1.0+60dbbe4 to easy-install.pth file

Installed /coc/scratch/sanisetty3/music_motion/VirtualMarker
Processing dependencies for virtualmarker==0.1.0+60dbbe4
Finished processing dependencies for virtualmarker==0.1.0+60dbbe4


## CIRCLE

In [6]:
import numpy as np
from scipy.spatial.transform import Rotation as R

In [7]:


%cd ../motion-diffusion-model/
import utils.rotation_conversions as geometry
from model.smpl import SMPL

smpl_model = SMPL().eval()
%cd ../motion_vqvae/

/coc/scratch/sanisetty3/music_motion/motion-diffusion-model
/coc/scratch/sanisetty3/music_motion/motion_vqvae


In [8]:
# !wget https://circledataset.s3.us-west-2.amazonaws.com/release/CIRCLE_movement.zip

In [9]:
# !unzip CIRCLE_movement.zip

In [10]:
from utils.rotation_conversions import axis_angle_to_matrix

In [43]:
mots =   glob("/srv/scratch/sanisetty3/music_motion/CIRCLE/*/*/*/*/*.npz")

In [52]:
mot1 = np.load(mo1ats[10] , allow_pickle = True)
transl = mot1['trans']
pose = mot1["poses"]

In [53]:
global_orient = pose[:, :3]
body_pose = pose[:, 3:66]
body_pose2 = np.concatenate((body_pose , body_pose[:,-6:]) , 1)

In [54]:
smpll = smpl_model.forward(
        global_orient=torch.from_numpy(R.from_rotvec(global_orient.reshape((-1,3))).as_matrix().reshape((-1,1,3,3))).float(),
        body_pose=torch.from_numpy(R.from_rotvec(body_pose2.reshape((-1,3))).as_matrix().reshape((-1,23,3,3))).float(),
        transl=torch.from_numpy(transl).float(),
    )

In [55]:
round(smpll["smpl"].shape[0]/6)

88

In [56]:
motion = torch.zeros(( round(smpll["smpl"].shape[0]/6) , 24,3))
ii = 0
for i in range(0,smpll["smpl"].shape[0],6):
    motion[ii] = smpll["smpl"][i]
    ii+=1
    

In [57]:
sample_render(motion[:,:22][None,...], "sample" , "/srv/scratch/sanisetty3/music_motion/Choreomaster/" , fps = 20)

render start


In [34]:
bvh = glob("/srv/scratch/sanisetty3/music_motion/CIRCLE/*/*/*/*/*.json")

In [35]:
for p in bvh:
    os.remove(p)

## MOYO

In [8]:
import pickle
import numpy as np
from scipy.spatial.transform import Rotation as R
from utils.rotation_conversions import axis_angle_to_matrix

In [9]:
with open('/srv/scratch/sanisetty3/music_motion/MOYO/mosh2/train/220923_yogi_body_hands_03596_Boat_Pose_or_Paripurna_Navasana_-a_stageii.pkl', 'rb') as f:
    x = pickle.load(f)


In [15]:
x.keys()

dict_keys(['fullpose', 'trans', 'stageii_debug_details', 'betas', 'markers_latent', 'latent_labels', 'marker_meta', 'markers_latent_vids', 'v_template_fname', 'stagei_debug_details'])

In [10]:


%cd ../motion-diffusion-model/
import utils.rotation_conversions as geometry
from model.smpl import SMPL

smpl_model = SMPL().eval()
%cd ../motion_vqvae/

/coc/scratch/sanisetty3/music_motion/motion-diffusion-model
/coc/scratch/sanisetty3/music_motion/motion_vqvae


In [42]:
transl = x['trans']
pose = x["fullpose"]

In [43]:
global_orient = pose[:, :3]
body_pose = pose[:, 3:66]
body_pose2 = np.concatenate((body_pose , body_pose[:,-6:]) , 1)

In [44]:
transl=torch.from_numpy(transl).float()
global_orient=torch.from_numpy(R.from_rotvec(global_orient.reshape((-1,3))).as_matrix().reshape((-1,1,3,3))).reshape(body_pose.shape[0] , -1).float()
body_pose=torch.from_numpy(R.from_rotvec(body_pose2.reshape((-1,3))).as_matrix().reshape((-1,23,3,3))).reshape(body_pose.shape[0] , -1).float()

In [47]:
np.save("" , torch.cat((transl , global_orient,body_pose) , 1).numpy())

torch.Size([892, 219])

In [13]:
smpll = smpl_model.forward(
        global_orient=torch.from_numpy(R.from_rotvec(global_orient.reshape((-1,3))).as_matrix().reshape((-1,1,3,3))).float(),
        body_pose=torch.from_numpy(R.from_rotvec(body_pose2.reshape((-1,3))).as_matrix().reshape((-1,23,3,3))).float(),
        transl=torch.from_numpy(transl).float(),
    )

In [18]:
sample_render(smpll["smpl"][:,:22][None,...], "sample3" , "/srv/scratch/sanisetty3/music_motion/Choreomaster/" , fps = 60)

render start
